In [1]:
import os
import json
import re

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
# from azure.search.documents.aio import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult
from openai.types.chat import ChatCompletion

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_CHAT_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_CHAT_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_CHAT_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [2]:
conversation_price = 0
conversation_ct = 0
conversation_pt = 0
conversation_tt = 0


def get_conversation_price(new_completion, model="gpt4o"):
    global conversation_price, conversation_ct, conversation_pt, conversation_tt
    USD_price_in_COP = 4100

    if model == "gpt4o-mini":
        in_price = 0.00000015
        out_price = 0.0000006
    if model == "gpt4o":
        in_price = 0.000005
        out_price = 0.000015
    if model == "llama-3-8b":
        in_price = 0.00000005
        out_price = 0.00000008
    if model == "llama-3-70b":
        in_price = 0.00000059
        out_price = 0.0000007
    if model == "haiku":
        in_price = 0.00000025
        out_price = 0.00000125

    ct = new_completion.usage.completion_tokens
    pt = new_completion.usage.prompt_tokens
    tt = new_completion.usage.total_tokens
    usd_total_price = (ct * out_price) + (pt * in_price)
    cop_total_price = usd_total_price * USD_price_in_COP
    conversation_price += cop_total_price
    # conversation_ct += ct
    # conversation_pt += pt
    # conversation_tt += tt
    # conversation_usd_total_price = (conversation_ct * out_price) + (
    #     conversation_pt * in_price
    # )
    # conversation_cop_total_price = conversation_usd_total_price * USD_price_in_COP

    print(
        f"💰 Conversation price: ${conversation_price} COP\n"
        f"   This message: ${cop_total_price} COP "
        f""
    )

    return

In [3]:
conversation = []  # Comment to keep conversation

In [4]:
# user_input = "¿Qué dice el artículo 103 del Código Penal?"  # 1
# user_input = "Cómo se ha efectuado esta ley en la jurisprudencia?"
# user_input = "¿Cuáles son las causales de agravación del homicidio?"  # 2
# user_input = "Si A le roba el celular a B, ¿qué delito comete?"  # 3
# user_input = (
#     "¿Cuál es la diferencia entre dolo eventual y culpa con representación?"  # 4
# )
# user_input = "¿Cuáles son las formas de autoría en derecho penal?"  # 5
# user_input = "¿Cuándo se consuma el hurto?"  # 6
# user_input = "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?"  # 7
# user_input = "¿Cuál es el término que tiene la Fiscalía para formular acusación?"  # 8
# user_input = "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?"  # 9
# user_input = "¿Se puede precluir una investigación antes de formular imputación?"  # 10
# user_input = (
#     "¿Cuáles son las causales de ausencia de responsabilidad en el derecho penal?"  # 11
# )
# user_input = "¿Cuáles son las causales genéricas de atenuación punitiva?"  # 12
# user_input = "¿Qué se entiende por “documento” en el derecho penal?"  # 13
# user_input = "¿Cuáles son las causales de agravación de la estafa?"  # 14
# user_input = "¿Se pueden realizar preguntas sugestivas en el contrainterrogatorio del juicio oral?"  # 15
# user_input = "¿Un juez debe ordenar la captura del procesado cuando anuncie sentido de fallo condenatorio?"  # 16
# user_input = (
#     "¿Cuáles son las fuentes de posición de garante según el código penal?"  # 17
# )
# user_input = "¿La búsqueda selectiva en base de datos requiere control previo?"  # 18
# user_input = "¿Cuándo procede la captura en flagrancia?"  # 19
# user_input = "Si alguien falsifica un documento y lo usa como medio para engañar a otra persona y llevarlo a que le pague algo que no le debe, ¿qué delito(s) comete?"  # 20

# user_input = "Cuál es la sanción por no presentar la declaración de renta a timepo?"
# user_input = "Qué es el derecho a la vida?"
# user_input = "En qué artículo de la constitución está consagrado el derecho a la vida?"
# user_input = "En qué artículo del código penal se contempla la definición de documento?"
# user_input = "¿Qué dice el artículo 14 del código penal?"
# user_input = "¿Qué dice el artículo 14 de la ley 599 del 2000?"
# user_input = "¿Qué dice el artículo 241 del código penal?"
# user_input = "¿Qué se entiende por “documento” en el código penal?"
# user_input = "¿Qué dice el artículo 249 del código penal?"
# user_input = "¿Qué dice el artículo 221 del código penal?"
# user_input = "¿En qué artículo se encuentra el delito de calumnia?"
# user_input = "Si B imputa falsamente a A un delito que no ha cometido ¿En qué conducta punible incurre?"
# user_input = "En qué delito incurre una persona que entre sin autorización en un sistema informático?"
# user_input = "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# user_input = "¿Qué delito comtió una persona que adquirió un bien cuyo origen inmediato es una actividad ilícita?"
# user_input = "¿Qué dice el artículo 323 del código penal?"
# user_input = "¿Qué artículo contiene el delito de lavado de activos en el código penal?"
# user_input = "¿Qué delito comete un juez que profiere una sentencia manifiestamente contraria a la ley?"
# user_input = "¿Dónde se considera realizada la conducta punible?"
# user_input = "Si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "Según el artículo 14 del código penal si A dispara a B en Barrranquilla pero B se muere en Cartagena, ¿Dónde se entiende realizado el homicidio?"
# user_input = "¿Qué se entiende por servido público en código penal?"
# user_input = "¿Cuáles son las formas de autoría para el código penal?"
# user_input = (
#     "¿Cuáles son las modalidades en las que se pueden cometer una conducta punible?"
# )
# user_input = "Si A transporta 5 miligramos de marihuana con fines de comercialización ¿Qué delito comete?"
# user_input = (
#     "¿Cuáles son las funciones de las medidas de seguridad según el código penal?"
# )
# user_input = "¿En qué artículo del código penal se mencionan los elementos de la conducta punible?"
# user_input = "¿Quiénes son partícipes para el derecho penal?"
# user_input = "¿Cuáles son los tipos de penas según el código penal?"
# user_input = "¿Cuáles son las medidas de aseguramiento privativas de la libertad?"
# user_input = "¿Cuáles son las medidas de aseguramiento privativas de la libertad?"
# user_input = "En qué sentencias se encuentran referenciades este artículo?"
user_input = "Dame una sentencia en la que específicamente se haya dicho que quien cobra un peaje es un servidor público a efectos de lo previsto en el artículo 20 del código penal"

print("🧑 User input:")
print(user_input)

🧑 User input:
Dame una sentencia en la que específicamente se haya dicho que quien cobra un peaje es un servidor público a efectos de lo previsto en el artículo 20 del código penal


In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_sources",
            "description": "Retrieve sources from the Azure AI Search index",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "Query string to retrieve documents from azure search eg: 'Health care plan'",
                    }
                },
                "required": ["search_query"],
            },
        },
    }
]

In [6]:
query_system_template = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.
    You have access to Azure AI Search index with 100's of documents.
    Generate a search query based on the conversation and the new question.
    Do not include cited source filenames and document names e.g info.txt or doc.pdf in the search query terms.
    Do not include any text inside [] or <<>> in the search query terms.
    Do not include any special characters like '+'.
    If the question is not in English, translate the question to English before generating the search query.
    If you cannot generate a search query, return just the number 0.
    """

query_few_shots = [
    {"role": "user", "content": "¿Qué dice el artículo 103 del Código Penal?"},
    {"role": "assistant", "content": '"Artículo 103" Ley 599 de 2000'},
    {
        "role": "user",
        "content": "¿Cuáles son las causales de agravación del homicidio?",
    },
    {
        "role": "assistant",
        "content": '"Causales de agravación del homicidio" "Ley 599 de 2000"',
    },
    {"role": "user", "content": "Si A le roba el celular a B, ¿qué delito comete?"},
    {"role": "assistant", "content": '"Hurto" "Ley 599 de 2000" "Celular"'},
    {
        "role": "user",
        "content": "¿En qué artículo de la Constitución Política se consagra el principio de dignidad humana?",
    },
    {
        "role": "assistant",
        "content": '"Principio de dignidad humana" "Constitución Política de Colombia"',
    },
    {"role": "user", "content": "¿Cuáles son las formas de autoría en derecho penal?"},
    {
        "role": "assistant",
        "content": 'Formas de autoría en derecho penal "Ley 599 de 2000"',
    },
    {"role": "user", "content": "¿Cuándo se consuma el hurto?"},
    {"role": "assistant", "content": '"Hurto' 'consumación" "Ley 599 de 2000"'},
    {
        "role": "user",
        "content": "¿Las personas jurídicas pueden ser sujetos pasivos de los delitos de injuria y calumnia?",
    },
    {
        "role": "assistant",
        "content": '"Personas jurídicas" "sujetos pasivos" "delitos de injuria y calumnia"',
    },
    {
        "role": "user",
        "content": "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?",
    },
    {
        "role": "assistant",
        "content": '"Archivo de investigación" "legítima defensa" "Ley 906 de 2004"',
    },
    {"role": "user", "content": "¿Cuáles son los requisitos de la denuncia?"},
    {"role": "assistant", "content": '"Requisitos de la denuncia" Ley 906 de 2004'},
]

query_user_content_template = f"Generate search query for: {user_input}."

query_messages = [
    {"role": "system", "content": query_system_template},
    *query_few_shots,
    # *conversation[:5],
    {"role": "user", "content": query_user_content_template},
]

In [7]:
query_completion = openai_client.chat.completions.create(
    messages=query_messages,
    model=os.getenv("AZURE_CHAT_OPENAI_DEPLOYMENT"),
    temperature=0.0,
    n=1,
    tools=tools,
    max_tokens=100,
    tool_choice="auto",
)
get_conversation_price(query_completion)
print(json.dumps(query_messages, indent=4))
print("==============================")
print(query_completion)

💰 Conversation price: $13.981000000000002 COP
   This message: $13.981000000000002 COP 
[
    {
        "role": "system",
        "content": "Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.\n    You have access to Azure AI Search index with 100's of documents.\n    Generate a search query based on the conversation and the new question.\n    Do not include cited source filenames and document names e.g info.txt or doc.pdf in the search query terms.\n    Do not include any text inside [] or <<>> in the search query terms.\n    Do not include any special characters like '+'.\n    If the question is not in English, translate the question to English before generating the search query.\n    If you cannot generate a search query, return just the number 0.\n    "
    },
    {
        "role": "user",
        "content": "\u00bfQu\u00e9 dice el art\u00edculo 103 del C\u00f3digo Penal?"
    },
    {
    

In [8]:
def get_search_query(chat_completion: ChatCompletion, user_query: str):
    NO_RESPONSE = "0"
    response_message = chat_completion.choices[0].message

    if response_message.tool_calls:
        for tool in response_message.tool_calls:
            if tool.type != "function":
                continue
            function = tool.function
            if function.name == "search_sources":
                arg = json.loads(function.arguments)
                search_query = arg.get("search_query", NO_RESPONSE)
                if search_query != NO_RESPONSE:
                    return search_query
    elif query_text := response_message.content:
        if query_text.strip() != NO_RESPONSE:
            return query_text

    return user_query

In [9]:
query_text = get_search_query(query_completion, user_input)
# query_text = "sentencias referencian artículo 103 del Código Penal"
print("🔎 Query for:")
print(query_text)

🔎 Query for:
sentencia cobro peaje servidor público artículo 20 código penal


In [10]:
query_vectors = embeddings_client.generate_embeddings(content=query_text)

In [11]:
results = search_client.search(
    search_text=query_text,
    top=7,
    query_type=QueryType.SEMANTIC,
    vector_queries=[
        {
            "vector": query_vectors,
            "k": 7,
            "fields": "content_vector",
            "kind": "vector",
        }
    ],
    semantic_configuration_name="none",
    query_caption="extractive|highlight-false",
    scoring_profile="ariel-alberto",
    search_fields=["content"],
)

results_list = list(results)

sources = ""

for document in results_list:
    # print(document)
    doc_formatted = {
        # "score": document["@search.score"],
        # "rerank": document["@search.reranker_score"],
        "id": document["id"],
        "page": document["page"],
        "title": document["title"],
        "content": document["content"],
        "external_id": document["external_id"],
        "author": document["author"],
        "keywords": document["keywords"],
        "category": document["category"],
        "year": document["year"],
    }
    sources += (
        f"\n{document['title']} - Página {document['page']}: {document['content']}"
    )

print("🔎 Query for:")
print(query_text)
print("=============================")
print(sources)

🔎 Query for:
sentencia cobro peaje servidor público artículo 20 código penal

CC-T258-1995 - Página 7: 4.1. En el caso subjudice el peticionario alega la vulneración del derecho a la
educación, en razón de que para poder asistir a clases al Colegio Instituto San
Clemente, tanto él como sus compañeros deben sufragar el costo del peaje del
vehículo de servicio público puesto a su disposición por el municipio para
permitirles su desplazamiento diario al plantel educativo. Dado que dicho
peticionario ni sus compañeros cuentan con recursos económicos suficientes, se
afirma que muchas veces no han podido pagar el costo del peaje, con el
resultado de que han dejado de asistir a clases, lo cual ha impedido el ejercicio
del derecho a la educación.
Expresa el juzgado que en el presente caso se viola el derecho a la igualdad al
imponer a personas que viven en el campo, donde las condiciones son mucho
mas difíciles que en la ciudad, un esfuerzo económico, el pago de un peaje, para
que sus hijos pu

In [12]:
response_system_template = f'Eres Ariel, un asistente para la investigación legal. \n\n    Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas. Solamente si la respuesta no está en las fuentes proporcionadas, responde “No encuentro información con esos términos, ¿puedes reformular tu consulta?”. Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>).\n    \n    Cada fuente tiene un nombre seguido por dos puntos y la información real, siempre incluye el nombre de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. Asegúrate de capturar todo antes de ".pdf", por ejemplo: [SFC - Títulos valores electrónicos. Pagarés. Depósito centralizado de valores. Exigencia Concepto 2020086426-003 del 24 de junio de 2020 id2020086426.pdf]. No dividas los nombres de archivo por ninguna razón, ya que todos deben terminar en .pdf.\n\n    Sé lo más detallado y preciso posible en tus respuestas. Cita el máximo número de fuentes posible, sin salirte del tema. En caso de encontrar información contradictoria, señálala y sugiere una posible causa. Expresa toda la información que encuentres en las fuentes proporcionadas. Solamente si la respuesta no está en las fuentes proporcionadas, responde “No encuentro información con esos términos, ¿puedes reformular tu consulta?”. Al incluir títulos en tu respuesta, usa formato html (ej: titulos, <strong>).\n    \n    Cada fuente tiene un nombre seguido por dos puntos y la información real, siempre incluye el nombre de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, lista cada fuente por separado, por ejemplo [info1.txt] [info2.pdf]. El formato de algunas fuentes puede incluir acentos, puntos o guiones. Asegúrate de capturar todo antes de ".pdf", por ejemplo: [SFC - Títulos valores electrónicos. Pagarés. Depósito centralizado de valores. Exigencia Concepto 2020086426-003 del 24 de junio de 2020 id2020086426.pdf]. No dividas los nombres de archivo por ninguna razón, ya que todos deben terminar en .pdf. Escribe la respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta.\n\n        '

response_few_shots = []

response_user_content_template = f"{user_input}\n\nSources:\n{sources}"
conversation += [{"role": "user", "content": response_user_content_template}]


response_messages = [
    {"role": "system", "content": response_system_template},
    *response_few_shots,
] + conversation[:5]

# print(json.dumps(conversation, indent=4))
# print("==================")
print(json.dumps(response_messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres Ariel, un asistente para la investigaci\u00f3n legal. \n\n    S\u00e9 lo m\u00e1s detallado y preciso posible en tus respuestas. Cita el m\u00e1ximo n\u00famero de fuentes posible, sin salirte del tema. En caso de encontrar informaci\u00f3n contradictoria, se\u00f1\u00e1lala y sugiere una posible causa. Expresa toda la informaci\u00f3n que encuentres en las fuentes proporcionadas. Solamente si la respuesta no est\u00e1 en las fuentes proporcionadas, responde \u201cNo encuentro informaci\u00f3n con esos t\u00e9rminos, \u00bfpuedes reformular tu consulta?\u201d. Al incluir t\u00edtulos en tu respuesta, usa formato html (ej: titulos, <strong>).\n    \n    Cada fuente tiene un nombre seguido por dos puntos y la informaci\u00f3n real, siempre incluye el nombre de la fuente para cada hecho que uses en la respuesta. Todas las fuentes son PDFs. Utiliza corchetes para referenciar la fuente, por ejemplo [info1.pdf]. No combines fuentes, 

In [13]:
response = openai_client.chat.completions.create(
    messages=response_messages,
    model=os.getenv("AZURE_CHAT_OPENAI_DEPLOYMENT"),
    temperature=0.2,
    n=1,
    max_tokens=1024,
)
get_conversation_price(response)
conversation += [{"role": "assistant", "content": response.choices[0].message.content}]

💰 Conversation price: $92.33200000000002 COP
   This message: $78.35100000000001 COP 


In [14]:
print(json.dumps(conversation, indent=4))

[
    {
        "role": "user",
        "content": "Dame una sentencia en la que espec\u00edficamente se haya dicho que quien cobra un peaje es un servidor p\u00fablico a efectos de lo previsto en el art\u00edculo 20 del c\u00f3digo penal\n\nSources:\n\nCC-T258-1995 - P\u00e1gina 7: 4.1. En el caso subjudice el peticionario alega la vulneraci\u00f3n del derecho a la\neducaci\u00f3n, en raz\u00f3n de que para poder asistir a clases al Colegio Instituto San\nClemente, tanto \u00e9l como sus compa\u00f1eros deben sufragar el costo del peaje del\nveh\u00edculo de servicio p\u00fablico puesto a su disposici\u00f3n por el municipio para\npermitirles su desplazamiento diario al plantel educativo. Dado que dicho\npeticionario ni sus compa\u00f1eros cuentan con recursos econ\u00f3micos suficientes, se\nafirma que muchas veces no han podido pagar el costo del peaje, con el\nresultado de que han dejado de asistir a clases, lo cual ha impedido el ejercicio\ndel derecho a la educaci\u00f3n.\nExpres

In [15]:
ct_price = 0.000005
pt_price = 0.000015
total_price = (response.usage.completion_tokens * ct_price) + (
    response.usage.prompt_tokens * pt_price
)
dolar_to_cop = 4089
print(
    f"CT:{response.usage.completion_tokens * ct_price} tokens - PT: {response.usage.prompt_tokens * pt_price} tokens"
)
print(
    f"CT: ${response.usage.completion_tokens * ct_price} USD - PT: {response.usage.prompt_tokens * pt_price} USD \n"
    f"Price: ${total_price} USD\n"
    f"Price: ${total_price * dolar_to_cop} COP"
)
print("🙎‍♂️ User: ")
print(user_input)
print("")
print("=============================================")
print("🔎 Query for: ")
print(query_text)
print("=============================================")
print("")
print(response.choices[0].message.content)
print("")
print("")
print(response)
# dónde se considera realizada la conducta punible Ley 599 de 2000
# lugar de realización de la conducta punible Ley 599 de 2000
# lugar de realización de la conducta punible Ley 599 de 2000

# homicidio lugar de realización disparo en un lugar muerte en otro Ley 599 de 2000
# homicidio lugar de realización disparo en una ciudad muerte en otra

CT:7.500000000000001e-05 tokens - PT: 0.056655000000000004 tokens
CT: $7.500000000000001e-05 USD - PT: 0.056655000000000004 USD 
Price: $0.05673 USD
Price: $231.96897 COP
🙎‍♂️ User: 
Dame una sentencia en la que específicamente se haya dicho que quien cobra un peaje es un servidor público a efectos de lo previsto en el artículo 20 del código penal

🔎 Query for: 
sentencia cobro peaje servidor público artículo 20 código penal

No encuentro información con esos términos, ¿puedes reformular tu consulta?


ChatCompletion(id='chatcmpl-A9f18HBHvR8Oy4GorDofM97fzG8IT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='No encuentro información con esos términos, ¿puedes reformular tu consulta?', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filte